## ICESat-2 SlideRule Download

This code is adapted from Sliderule Python example code. It is designed to calculate ATL06 elevations from ALT03 points and save those elevations as a CSV. 

Activate sliderule enviroment to run

In [1]:
import sys
import logging
import concurrent.futures
import time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
from sliderule import icesat2
from sliderule import sliderule

## SlideRule Configuration

In [2]:
# Configure ICESat-2 API
icesat2.init("icesat2sliderule.org", verbose=False)

## Specify Region of Interest

In [5]:
# Specify region of interest from geojson
shapefile = gpd.read_file("boundDCEW/boundDCEW.shp")


In [7]:
region = icesat2.toregion(shapefile)["poly"]
region

[{'lon': 2324729.4288565014, 'lat': 1389613.3876734977},
 {'lon': 2330951.081373454, 'lat': 1390620.8240924913},
 {'lon': 2331047.773344156, 'lat': 1390646.0362778297},
 {'lon': 2331384.388417789, 'lat': 1390823.5242257472},
 {'lon': 2331427.2303362493, 'lat': 1390848.0053220058},
 {'lon': 2331610.8385582305, 'lat': 1391160.1392993778},
 {'lon': 2332192.5382502167, 'lat': 1394607.0648326157},
 {'lon': 2332192.5382502167, 'lat': 1394662.0480679092},
 {'lon': 2332183.374377668, 'lat': 1394762.8506659525},
 {'lon': 2331981.769181581, 'lat': 1395752.5489012902},
 {'lon': 2331926.7859462867, 'lat': 1395844.1876267896},
 {'lon': 2331257.8232501824, 'lat': 1396623.1167934865},
 {'lon': 2331074.545799195, 'lat': 1396788.0664993771},
 {'lon': 2331019.562563897, 'lat': 1396797.230371926},
 {'lon': 2327915.3904704927, 'lat': 1395301.9722364955},
 {'lon': 2326359.096224969, 'lat': 1394056.3319696062},
 {'lon': 2325898.9867378525, 'lat': 1393499.6562938332},
 {'lon': 2324409.3247291325, 'lat': 1389

In [6]:
shapefile

,Id,geometry
0,0,"POLYGON ((2324406.857 1389750.780, 2324409.325..."


In [8]:
# Read geojson with geopandas
pregion = gpd.read_file(shapefile)
pregion.plot();

AttributeError: 'GeoDataFrame' object has no attribute 'startswith'

In [9]:
# Prepare coordinate lists for plotting the region of interest polygon
region_lon = [e["lon"] for e in region]
region_lat = [e["lat"] for e in region]

## Specify parameters for ATL06-SR (Sliderule) processing

In [10]:
# Build ATL06 Request
parms = {
    "poly": region,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "ats": 10.0,
    "cnt": 10,
    "len": 40.0,
    "res": 20.0,
    "maxi": 1
}

## Calculate ATL06-SR Elevations from ATL03 Photons using SlideRule

In [13]:
# Latch Start Time
perf_start = time.perf_counter()

# Request ATL06 Data
atl06_sr = icesat2.atl06p(parms)

# Latch Stop Time
perf_stop = time.perf_counter()

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
print("Reference Ground Tracks: {}".format(atl06_sr["rgt"].unique()))
print("Cycles: {}".format(atl06_sr["cycle"].unique()))
print("Received {} elevations".format(atl06_sr.shape[0]))

Completed in 1.417 seconds of wall-clock time
Received 0 elevations


## Plot ATL06-SR Results

In [14]:
f, ax = plt.subplots()
ax.set_title("ATL06-SR Points")
ax.set_aspect('equal')
atl06_sr.plot(ax=ax, column='h_mean', cmap='inferno', s=0.1)
ax.plot(region_lon, region_lat, linewidth=1, color='g');

/Users/karinazikan/opt/anaconda3/envs/sliderule/lib/python3.8/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


ValueError: cannot convert float NaN to integer

<Figure size 432x288 with 1 Axes>